> **DO NOT EDIT IF INSIDE annadl_f19 folder**


# Week 4: Convolutional Neural Networks

We learned about feedforward neural networks over the last two weeks, along with most of the fundamental maths, algorithms, tricks, etc. that go into training them. Knowing how signal propagates forward, how gradients flow backwards and how weights get updates, we are now ready to release our imagination and create some deep neural networks that can do remarkable things. We will first focus on one of the most **powerful architectures**, the Convolutional Neural Network (CNN), which has completely revolutionalized image recognition in science and tech.

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist

Using TensorFlow backend.


## Exercises

The problem with feed forward images when classifying images is that the input layer looks at the whole image at once. Each input neuron thus becomes associated with a *specific* pixel, and as the network learns it may grow to expect a certain signal to emerge at that particular pixel. But if you imagine an image of a cat, features, like the whiskers, which make it cat-like, are not bound to a specific *place* in the image – they can be in the centre, top corner, or any other place.

CNNs solve this problem, with what's called *convolutional layers*. A convolutional input layer, for example, doesn't have a single weight for each input value. Instead, it has one or more much smaller *filters*, each one a set of input neurons (often $3 \times 3 \times d$ where $d$ is the depth of the input image (3 if it's an RGB image, 1 if its black and white)) that gets *convolved* across the input image to produce a new image, called an *activation map*. [Here](https://github.com/vdumoulin/conv_arithmetic) are some nice gifs that illustrate different ways one can convolve a weight matrix across an input image.

#### Pen and paper

To get your intuition for computations on input data in CNNs fine-tuned, I have a few small quizzes for you. First, we'll consider the size of the parameter space.

> **Ex. 4.1.1**: Imagine you have a CNN with just one convolutional layer with a single filter. All it does, is take an input image and produce an activation map. The dimensionality of the filter in your convolutional layer is $5 \times 5 \times 3$. How many weights (or *parameters*) are there in this model?
>
> *Hint*: Don't forget the bias!

Here's the formula for computing the size of the activation map resulting from a convolution. 
If you have a filter that is $F$ wide, your input image is $W_0$ wide, you are padding the edges by
$P$ pixels and your stride is $S$, the resulting image will have width/height:

$$ W_1 = \frac{W_0 - F + 2P}{S} + 1 $$

> **Ex. 4.1.2**: You input an image of dimensions $28 \times 28 \times 3$, use a padding of 2, a stride of 1,
and then slide your $5 \times 5 \times 3$ filter across the image. What is the dimensionality of the resulting activation map?

> **Ex. 4.1.3**: Let's say you now want to use a stride of 2, instead of 1. What problem does this immediately cause?
How can we solve it?

*Maxpooling* is a method used a lot in CNNs, which downsamples the size of an activation map. It is used primarily to reduce the amount of parameters and computations needed in the network, and to avoid overfitting. Here's an illustration of how it works:

![img](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

Typically, for each $2 \times 2$ square in your activation map, you pick the largest value in that square. You do this independently for every depth slice in your activation map.

**Note:** In Keras, the dimension of data is a little different from what you may expect. The first index,
indexes datapoints, the second and third are the dimensions of your images, and the last is number of channels. So if
you have a batch of data containing 100 datapoints, each one an RGB image (so 3 channels: red, green, blue)
with resolution $128 \times 128$, then the dimensionality of your input data is (100, 128, 128, 3).

> **Ex. 4.1.4**: Given the activation map below, what is the corresponding activation map after maxpooling ($2 \times 2$ filter, stride 2)? Run it through a Keras maxpooling layer (check out the docs), and report the dimensionality.

In [23]:
a = np.random.random(size=(10, 28, 28, 1))
activation_map = keras.backend.variable(a)

#### CNNs in Keras

For example sake, I have implemented a single conv. layer neural network Keras below.

In [30]:
model = Sequential([
    Conv2D(filters=10, kernel_size=3, strides=(1, 1), padding='valid'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(10)
])

In the following exercise you will use the MNIST dataset again. In the cell below I have written some code to prepare it somewhat. For your own sake, try to understand what's happening.

In [64]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape data so it has a channel dimension
rows, cols = x_train.shape[-2:]
x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)

# Convert pixel intensities to values between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
    
# Convert target vectors to one-hot encoding
num_classes = len(set(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

> **Ex. 4.1.5**: Implement Nielsen's [last convolutional neural network](http://neuralnetworksanddeeplearning.com/chap6.html#exercise_683491)
(the one with two convolutional layers and dropout), and score an accuracy higher than 98%. It doesn't have to be
fully identical, but his solution is pretty great, so getting close is a cheap way to score a high accuracy.
>
> *Hint:* [here](https://keras.io/examples/mnist_cnn/) is an example of a CNN in Keras. Not an invitation to copy, I'm only linking to this so you have a reference for how it's done, remember you want something similar to Nielsen's.

The MNIST dataset is a great benchmark because it allows us to directly measure the performance of a neural
network against the human eye. But often, the association between datapoint and label is not as clear, yet we
still want to do prediction.

I went on [Kaggle.com](www.kaggle.com) and looked for some *harder* datasets. At the time of writing there's
[a competetion](https://www.kaggle.com/c/petfinder-adoption-prediction) where you can win up to $\$$25.000 by
predicting how quickly pets get adopted (on a pet-adoption site) based on meta data and images of the pets. From that competetion, I prepared a dataset of $128 \times 128$ images along with adoption time (lower is better, see description on Kaggle). In the actual competetion people are using more data sources than just image, but for
now, let's see how well we can do with just that. Here are links to download the prepared datasets ([train](https://www.dropbox.com/s/zjno7wpmlurw8bo/adoptability_data.zip?dl=0) // [test](https://www.dropbox.com/s/9m93dzhjc9f9jrr/adoptability_test.zip?dl=0)).

> **Ex. 4.1.6**: Load the prepared dataset and build a CNN that predicts adoption-time category from the image.
Leave out the last 1000 datapoints from training and save them for validation. You have complete freedom in how
you want to construct the neural network (choice of cost function, inner architecture, training hyperparameters,
etc.). Report an accuracy on the test set (last 1000 points) that's higher than the baseline (0.273). Using your
adoptability predictor, **visualize the 5 most adoptable dogs**.
>
> As an extra bonus, I also included a test dataset with no corresponding target array. You can take your model and try to predict the test data, and format it similar to the `sample_submission.csv` file and submit your predictions to the competetion to get an estimate of the general performance of your model. **Let me know what you score if you do that**. And if you choose to join the competetion and try to win the money, I'd also like to hear about your progress. You can keep the money.
>
> Also, note that to get high accuracy, you need to make a network that is bigger than what you had in the previous exercise. This will take a long time to train until convergence (hours maybe). If you have a computer with a small
CPU you will have to make do with a smaller neural network, and thus a lower accuracy. But you should still be able to score above the baseline.

If you download the data and put in the same folder as this notebook,
then the below code will prepare your data nicely.

In [3]:
# Load data
x = np.load('/Users/ulfaslak/Downloads/adoptability_data/x.npy').astype('float32')
y = np.load('/Users/ulfaslak/Downloads/adoptability_data/y.npy').astype(int)

# Transform images from 0-255 to 0-1 values
x /= 255

# Reshape data
x = x.reshape(-1, 128, 128, 3)
num_classes = len(set(y))
y = keras.utils.to_categorical(y, num_classes)

# Split into train and test
x_test = x[-1000:]
y_test = y[-1000:]
x_train = x[:-1000]
y_train = y[:-1000]

In [100]:
# Baseline
print(max(y.sum(0))/y.sum())

0.27300027
